In [1]:
import numpy as np
#import tensorflow as tf
from glob import glob
import os
from os.path import join,exists,basename
import json
import random
import cv2 as cv
from natsort import natsorted
import tensorflow as tf
%matplotlib inline

2023-02-19 19:13:16.836574: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-19 19:13:17.471563: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/cvlab/anaconda3/envs/vision/lib/python3.10/site-packages/cv2/../../lib64:/opt/ros/foxy/opt/yaml_cpp_vendor/lib:/opt/ros/foxy/opt/rviz_ogre_vendor/lib:/opt/ros/foxy/lib/x86_64-linux-gnu:/opt/ros/foxy/lib:/usr/local/cuda-11.7/lib64${LD_LIBRARY_PATH:+:}
2023-02-19 19:13:17.471749: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvin

: 

In [2]:
### Information about directory src and target 
dires={'raw_videos':join('dataset_raw','smarthome_raw','alphapose','CO','raw_video'),
'img_ids':join('datasets_features','smarthome','alphapose','CO','img_id')}

In [3]:
rgb_target_dires={'co':join('datasets_features','smarthome','rgb','CO'),
'cd':join('datasets_features','smarthome','CD')
}

In [5]:
seq_len=32
w,h=640,480
co_classnames=natsorted(os.listdir(dires['img_ids']))

In [6]:
co_classnames

['Drink.Frombottle', 'Drink.Fromcan', 'Drink.Fromcup']

In [7]:
imgs_ids_test=glob(join(dires['img_ids'],co_classnames[0])+'/*.npy')[0]

In [8]:
## Samples Frames  Function 

def get_sampled_frames(video_dire,img_id_list):
    sampled_frames=[]
    cap=cv.VideoCapture(video_dire)
    for frame_number in img_id_list:
        cap.set(cv.CAP_PROP_POS_FRAMES, frame_number)
        sucess,frame=cap.read()
        sampled_frames.append(frame)

    sampled_frames=np.array(sampled_frames)

    return sampled_frames

In [9]:
## Pretrained_model


def resnet(sampled_features):
    pass

def get_densenet121(resized_height=h,resized_width=w):
    ## defining the DenseNet121 model pretrained with imagenet dataset
    feature_extractor=tf.keras.applications.DenseNet121(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(resized_height,resized_width,3),
    pooling='max',

)
    ##Defining the input size for single image                    
    inputs=tf.keras.Input((resized_height,resized_width,3))
    preprocessed=tf.keras.applications.densenet.preprocess_input(inputs)

    ## Give features for each image
    outputs=feature_extractor(preprocessed)
    ## defining feature extractor model
    model=tf.keras.Model(inputs,outputs,name='feature_extractor')
    return model

densenet_model=get_densenet121()   

2023-01-26 10:48:33.887081: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 10:48:33.912350: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 10:48:33.912476: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-01-26 10:48:33.912933: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [10]:
features=[]
labels=[]
for idx,classname in enumerate(co_classnames):
    img_id_dires=glob(join(dires['img_ids'],classname)+'/*.npy')
    for img_id_dire in img_id_dires:
        filename=basename(img_id_dire)
        video_filename=filename[:-3]+'mp4'
        video_dire=join(dires['raw_videos'],classname,video_filename)
        img_id=np.load(img_id_dire)
        img_id=[int(i.split('.')[0])for i in img_id]
        sampled_frames=get_sampled_frames(video_dire,img_id)
        feature=densenet_model.predict(sampled_frames)
        features.append(feature)
        labels.append(idx)

        target_folder=join(rgb_target_dires['co'],'noncrop','densenet121',classname)

        if not exists(target_folder):
            os.makedirs(target_folder)

        target_filedire=join(target_folder,filename)

        np.save(target_filedire,feature)


        
features=np.array(features)
labels=np.array(labels)


2023-01-26 10:48:39.597640: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8700


1/1 [==============================] - 0s 42ms/step


In [ ]:
features.shape

In [11]:
from sklearn.model_selection import train_test_split


In [12]:
train_features, test_features, train_labels, test_labels=train_test_split(features, labels, test_size=0.25, random_state=101,stratify=labels)

In [13]:

saved_dires_train=join(rgb_target_dires['co'],'noncrop','densenet121','features','train')
if not os.path.exists(saved_dires_train):
    os.makedirs(saved_dires_train)

saved_dires_test=join(rgb_target_dires['co'],'noncrop','densenet121','features','test')
if not os.path.exists(saved_dires_test):
    os.makedirs(saved_dires_test)

In [14]:
np.save(join(saved_dires_train,'features.npy'),train_features)
np.save(join(saved_dires_train,'labels.npy'),train_labels)

np.save(join(saved_dires_test,'features.npy'),test_features)
np.save(join(saved_dires_test,'labels.npy'),test_labels)

: 